In [1]:
import pandas as pd
import sqlite3

# Load Telco dataset
df = pd.read_csv(
    r"C:\Users\satis\OneDrive\Desktop\Python final project\housing_app_fall25\data\raw\Telco-Customer-Churn.csv"
)

print(df.shape)
print(df.columns)
df.head()


(7043, 21)
Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [2]:
customers = df[
    ["customerID", "gender", "SeniorCitizen", "Partner", "Dependents"]
].copy()

customers.rename(columns={
    "customerID": "customer_id",
    "SeniorCitizen": "senior_citizen"
}, inplace=True)


In [5]:
services = df[
    ["customerID", "PhoneService", "MultipleLines", "InternetService",
     "OnlineSecurity", "OnlineBackup", "DeviceProtection",
     "TechSupport", "StreamingTV", "StreamingMovies"]
].copy()

services.rename(columns=lambda x: x.lower(), inplace=True)
services.rename(columns={"customerid": "customer_id"}, inplace=True)

accounts = df[
    ["customerID", "Contract", "PaperlessBilling", "PaymentMethod",
     "tenure", "MonthlyCharges", "TotalCharges", "Churn"]
].copy()

accounts.rename(columns={
    "customerID": "customer_id",
    "MonthlyCharges": "monthly_charges",
    "TotalCharges": "total_charges",
    "PaperlessBilling": "paperless_billing",
    "PaymentMethod": "payment_method"
}, inplace=True)

# Convert TotalCharges to numeric
accounts["total_charges"] = pd.to_numeric(
    accounts["total_charges"], errors="coerce"
)

conn = sqlite3.connect("telco_churn.db")

customers.to_sql("Customers", conn, index=False, if_exists="replace")
services.to_sql("Services", conn, index=False, if_exists="replace")
accounts.to_sql("Accounts", conn, index=False, if_exists="replace")

print("✅ Telco tables saved to SQLite successfully!")


query = """
SELECT
    a.customer_id,
    a.tenure,
    a.monthly_charges,
    a.total_charges,
    a.contract,
    a.payment_method,
    a.paperless_billing,
    a.churn,

    c.gender,
    c.senior_citizen,
    c.partner,
    c.dependents,

    s.phoneservice,
    s.multiplelines,
    s.internetservice,
    s.onlinesecurity,
    s.onlinebackup,
    s.deviceprotection,
    s.techsupport,
    s.streamingtv,
    s.streamingmovies

FROM Accounts a
JOIN Customers c ON a.customer_id = c.customer_id
JOIN Services s ON a.customer_id = s.customer_id
LIMIT 10;
"""

df_joined = pd.read_sql_query(query, conn)
df_joined.head()




✅ Telco tables saved to SQLite successfully!


,customer_id,tenure,monthly_charges,total_charges,Contract,payment_method,paperless_billing,Churn,gender,senior_citizen,...,Dependents,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies
0,7590-VHVEG,1,29.85,29.85,Month-to-month,Electronic check,Yes,No,Female,0,...,No,No,No phone service,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,34,56.95,1889.50,One year,Mailed check,No,No,Male,0,...,No,Yes,No,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,2,53.85,108.15,Month-to-month,Mailed check,Yes,Yes,Male,0,...,No,Yes,No,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,45,42.30,1840.75,One year,Bank transfer (automatic),No,No,Male,0,...,No,No,No phone service,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,2,70.70,151.65,Month-to-month,Electronic check,Yes,Yes,Female,0,...,No,Yes,No,Fiber optic,No,No,No,No,No,No


In [11]:
import sys
!{sys.executable} -m pip install ydata-profiling

  Using cached ydata_profiling-4.18.0-py2.py3-none-any.whl.metadata (22 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached visions-0.8.1-py3-none-any.whl.metadata (11 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached multimethod-1.12-py3-none-any.whl.metadata (9.6 kB)
  Using cached typeguard-4.4.4-py3-none-any.whl.metadata (3.3 kB)
  Using cached ImageHash-4.3.2-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached dacite-1.9.2-py3-none-any.whl.metadata (17 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached puremagic-1.30-py3-none-any.whl.metadata (5.8 kB)
Using cached ydata_profiling-4.18.0-py2.py3-none-any.whl (398 kB)
Using cached ImageHash-4.3.2-py2.py3-none-any.whl (296 kB)
Using cached dacite-1.9.2-py3-none-any.whl (16 kB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ------------------------------------

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(
    r"C:\Users\satis\OneDrive\Desktop\Python final project\housing_app_fall25\data\raw\Telco-Customer-Churn.csv"
)

print(f"Dataset shape: {df.shape}")
print(df.columns)
df.head()

# Target
y = df["Churn"].map({"Yes": 1, "No": 0})

# Features
X = df.drop(columns=["Churn", "customerID"])

print("Churn distribution:\n", y.value_counts(normalize=True))

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

profile = ProfileReport(
    df,
    title="Telco Customer Churn Profiling Report",
    explorative=True
)

profile.to_file("telco_churn_profile_report.html")
print("✅ Profiling report saved as telco_churn_profile_report.html")

numeric_cols = df.select_dtypes(include="number")

plt.figure(figsize=(12, 8))
sns.heatmap(
    numeric_cols.corr(),
    annot=True,
    fmt=".2f",
    cmap="coolwarm"
)
plt.title("Correlation Matrix – Telco Customer Churn")
plt.show()



Dataset shape: (7043, 21)
Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')
Churn distribution:
 Churn
0    0.73463
1    0.26537
Name: proportion, dtype: float64
Train shape: (5634, 19), Test shape: (1409, 19)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:01<00:00, 16.89it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Profiling report saved as telco_churn_profile_report.html


C:\Users\satis\AppData\Local\Temp\ipykernel_14332\2505013167.py:52: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
